# Delayed starts


In many applications, it is needed to 'delay' the start of particle advection. For example because particles need to be released at different times throughout an experiment. Or because particles need to be released at a constant rate from the same set of locations.

This tutorial will show how this can be done. We start with importing the relevant modules.


In [16]:
from datetime import timedelta

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from IPython.display import HTML
from matplotlib.animation import FuncAnimation

import parcels

First import a `FieldSet` (from the Argo example, in this case)


In [38]:
# Load the CopernicusMarine data in the Agulhas region from the example_datasets
example_dataset_folder = parcels.download_example_dataset(
    "CopernicusMarine_data_for_Argo_tutorial"
)

ds = xr.open_mfdataset(f"{example_dataset_folder}/*.nc", combine="by_coords")
ds.load()  # load the dataset into memory

fieldset = parcels.FieldSet.from_copernicusmarine(ds)

INFO: cf_xarray found variable 'uo' with CF standard name 'eastward_sea_water_velocity' in dataset, renamed it to 'U' for Parcels simulation.
INFO: cf_xarray found variable 'vo' with CF standard name 'northward_sea_water_velocity' in dataset, renamed it to 'V' for Parcels simulation.


Defining the initial times of particles is done when the `ParticleSet` is defined. Although `time` and `z` are optional arguments, it is good practice to define them explicitly to ensure expected behavior.

<div class="alert alert-info">

Note that the `repeatdt` argument used in previous versions of parcels is no longer supported as of v4
</div>

## Assigning each particle its own `time`


The simplest way to delay the start of a particle is to use the `time` argument for each particle


In [31]:
npart = 10  # number of particles to be released
lon = 32 * np.ones(npart)
lat = np.linspace(-31.5, -30.5, npart, dtype=np.float32)
# release every particle one hour later from the initial fieldset time
time = ds.time.values[0] + np.arange(0, npart) * np.timedelta64(1, "h")
z = np.repeat(ds.depth.values[0], npart)

pset = parcels.ParticleSet(
    fieldset=fieldset, pclass=parcels.Particle, lon=lon, lat=lat, time=time, z=z
)

Then we can execute the `pset` as usual


In [32]:
output_file = parcels.ParticleFile(
    "DelayParticle_time.zarr", outputdt=np.timedelta64(1, "h")
)

pset.execute(
    parcels.kernels.AdvectionRK4,
    runtime=np.timedelta64(24, "h"),
    dt=np.timedelta64(5, "m"),
    output_file=output_file,
)

INFO: Output files are stored in /Users/Gebruiker/Documents/GitHub/parcels/docs/examples_v3/DelayParticle_time.zarr
Integration time: 2024-01-01T23:00:11.128799232: 100%|██████████| 86400.0/86400.0 [00:01<00:00, 64348.53it/s]


And then finally, we can show a movie of the particles. Note that the southern-most particles start to move first.


In [ ]:
%%capture

ds_out_out = xr.open_zarr("DelayParticle_time.zarr", decode_times=False)

fig = plt.figure(figsize=(7, 5), constrained_layout=True)
ax = fig.add_subplot()

ax.set_ylabel("Latitude [deg N]")
ax.set_xlabel("Longitude [deg E]")
ax.set_xlim(31,33)
ax.set_ylim(-32,-30)

timerange = np.unique(ds_out_out["time"].values[np.isfinite(ds_out_out["time"])])

# Indices of the data where time = 0
time_id = np.where(ds_out_out["time"] == timerange[0])

sc = ax.scatter(ds_out_out["lon"].values[time_id], ds_out_out["lat"].values[time_id])

t = str(timerange[0].astype("timedelta64[h]"))
title = ax.set_title(f"Particles at t = {t}")


def animate(i):
    t = str(timerange[i].astype("timedelta64[h]"))
    title.set_text(f"Particles at t = {t}")

    time_id = np.where(ds_out_out["time"] == timerange[i])
    sc.set_offsets(np.c_[ds_out_out["lon"].values[time_id], ds_out_out["lat"].values[time_id]])


anim = FuncAnimation(fig, animate, frames=len(timerange), interval=100)

In [37]:
HTML(anim.to_jshtml())

## Release particles repeatedly at a set frequency using `np.tile`


The second method to delay the start of particle releases is to use the `repeatdt` argument when constructing a `ParticleSet`. This is especially useful if you want to repeatedly release particles from the same set of locations.


In [80]:
npart = 10  # number of particles to be released

lon_i = 32 * np.ones(npart)
lat_i = np.linspace(-31.5, -30.5, npart, dtype=np.float32)
# release particles at initial time
time_i = np.repeat(ds.time.values[0], npart)
z_i = np.repeat(ds.depth.values[0], npart)

# repeat release at frequency `repeatdt` for `nrepeat` different releases
repeatdt = np.timedelta64(6, 'h')
nrepeat = 3
lon = np.broadcast_to(lon_i,(nrepeat,npart))
lat = np.broadcast_to(lat_i,(nrepeat,npart))
time = np.broadcast_to(time_i,(nrepeat,npart)) + np.arange(0,nrepeat)[:,np.newaxis]*repeatdt
z = np.broadcast_to(z_i,(nrepeat,npart))

pset = parcels.ParticleSet(
    fieldset=fieldset, pclass=parcels.Particle, lon=lon, lat=lat, time=time, z=z
)

Now we again define an output file and execute the `pset` as usual.


In [68]:
output_file = parcels.ParticleFile(
    "DelayParticle_releasedt.zarr", outputdt=timedelta(hours=1)
)

pset.execute(
    parcels.kernels.AdvectionRK4,
    runtime=timedelta(hours=24),
    dt=timedelta(minutes=5),
    output_file=output_file,
)

INFO: Output files are stored in /Users/Gebruiker/Documents/GitHub/parcels/docs/examples_v3/DelayParticle_releasedt.zarr
Integration time: 2024-01-01T23:00:11.128799232: 100%|██████████| 86400.0/86400.0 [00:01<00:00, 75867.67it/s]


And we get an animation where a new particle is released every 3 hours from each start location


In [70]:
%%capture

ds_out = xr.open_zarr("DelayParticle_releasedt.zarr", decode_times=False)

fig = plt.figure(figsize=(7, 5), constrained_layout=True)
ax = fig.add_subplot()

ax.set_ylabel("Latitude [deg N]")
ax.set_xlabel("Longitude [deg E]")
ax.set_xlim(31,33)
ax.set_ylim(-32,-30)

timerange = np.unique(ds_out["time"].values[np.isfinite(ds_out["time"])])

# Indices of the data where time = 0
time_id = np.where(ds_out["time"] == timerange[0])

sc = ax.scatter(ds_out["lon"].values[time_id], ds_out["lat"].values[time_id])

t = str(timerange[0].astype("timedelta64[h]"))
title = ax.set_title(f"Particles at t = {t}")


def animate(i):
    t = str(timerange[i].astype("timedelta64[h]"))
    title.set_text(f"Particles at t = {t}")

    time_id = np.where(ds_out["time"] == timerange[i])
    sc.set_offsets(np.c_[ds_out["lon"].values[time_id], ds_out["lat"].values[time_id]])


anim = FuncAnimation(fig, animate, frames=len(timerange), interval=100)

In [71]:
HTML(anim.to_jshtml())

## Synced `time` in the output file

Note that, because the `outputdt` variable controls the Kernel-loop, all particles are written _at the same time_, even when they start at a non-multiple of `outputdt`.

For example, if your particles start at `time=[0, 1, 2]` and `outputdt=2`, then the times written (for `dt=1` and `endtime=4`) will be


In [86]:
outtime_expected = np.array(
    [[0, 2, 4], [2, 4, np.datetime64("NaT")], [2, 4, np.datetime64("NaT")]],
    dtype="timedelta64[h]",
)
print(outtime_expected)

[[    0     2     4]
 [    2     4 'NaT']
 [    2     4 'NaT']]


In [87]:
outtime_expected

array([[    0,     2,     4],
       [    2,     4, 'NaT'],
       [    2,     4, 'NaT']], dtype='timedelta64[h]')

In [83]:
outfilepath = "DelayParticle_nonmatchingtime.zarr"

pset = parcels.ParticleSet(
    fieldset=fieldset,
    pclass=parcels.Particle,
    lat=[-31] * 3,
    lon=[32] * 3,
    time=ds.time.values[0]+np.arange(3)*np.timedelta64(1,'h'),
    z=[0.5] * 3
)

output_file = parcels.ParticleFile(outfilepath, outputdt=np.timedelta64(2,'h'))
pset.execute(
    parcels.kernels.AdvectionRK4,
    endtime=ds.time.values[0]+np.timedelta64(4,'h'),
    dt=np.timedelta64(1,'h'),
    output_file=output_file,
)

# Note that we also need to write the final time to the file
output_file.write_latest_locations(pset, ds.time.values[0]+np.timedelta64(4,'h'))

INFO: Output files are stored in /Users/Gebruiker/Documents/GitHub/parcels/docs/examples_v3/DelayParticle_nonmatchingtime.zarr




Integration time: 2024-01-01T02:00:11.128799232: 100%|██████████| 14400.0/14400.0 [00:00<00:00, 305854.36it/s]


And indeed, the `time` values in the NetCDF output file are as expected


In [99]:
outtime_infile = xr.open_zarr(outfilepath).time.values[:]-ds.time.values[0]
print(outtime_infile.astype("timedelta64[h]"))

assert (
    outtime_expected[np.isfinite(outtime_expected)]
    == outtime_infile[np.isfinite(outtime_infile)]
).all()

[[1 3 4]
 [1 3 4]
 [2 3 4]]


ValueError: operands could not be broadcast together with shapes (7,) (9,) 

Now, for some applications, this behavior may be undesirable; for example when particles need to be analyzed at a same age (instead of at a same time). In that case, we recommend either changing `outputdt` so that it is a common divisor of all start times; or doing multiple Parcels runs with subsets of the original `ParticleSet` (e.g., in the example above, one run with the Particles that start at `time=[0, 2]` and one with the Particle at `time=[1]`). In that case, you will get two files:


In [102]:
np.array([[0, 2], [1, 3]]).astype('timedelta64[h]')

array([[0, 2],
       [1, 3]], dtype='timedelta64[h]')

In [112]:
for times in np.array([[0, 2], [1, 3]]).astype('timedelta64[h]'):
    pset = parcels.ParticleSet(
        fieldset=fieldset,
        pclass=parcels.Particle,
        lat=[-31] * len(times),
        lon=[32] * len(times),
        time=ds.time.values[0]+times,
        z=[0.5] * len(times)
    )
    output_file = parcels.ParticleFile(outfilepath, outputdt=np.timedelta64(2,'h'))
    pset.execute(
        parcels.kernels.AdvectionRK4,
        endtime=ds.time.values[0]+np.timedelta64(4,'h'),
        dt=np.timedelta64(1,'h'),
        output_file=output_file,
    )
    # Note that we also need to write the final time to the file
    output_file.write_latest_locations(pset,ds.time.values[0]+np.timedelta64(4,'h'))
    outtime_infile = xr.open_zarr(outfilepath).time.values[:]-ds.time.values[0]
    print(outtime_infile.astype("timedelta64[h]"))

INFO: Output files are stored in /Users/Gebruiker/Documents/GitHub/parcels/docs/examples_v3/DelayParticle_nonmatchingtime.zarr












Integration time: 2024-01-01T02:00:11.128799232: 100%|██████████| 14400.0/14400.0 [00:00<00:00, 243062.93it/s]
[[1 3 4]
 [2 3 4]]
INFO: Output files are stored in /Users/Gebruiker/Documents/GitHub/parcels/docs/examples_v3/DelayParticle_nonmatchingtime.zarr












Integration time: 2024-01-01T03:00:11.128799232: 100%|██████████| 10800.0/10800.0 [00:00<00:00, 271894.76it/s]
[[2 4]
 [3 4]]


## Adding new particles to a ParticleSet during runtime


In the examples above, all particles were defined at the start of the simulation. There are use-cases, though, where it is important to be able to add particles 'on-the-fly', during the runtime of a Parcels simulation.

Unfortuantely, Parcels does not (yet) support adding new particles _within_ a `Kernel`. A workaround is to temporarily leave the `execution()` call to add particles via the `ParticleSet.add()` method, before continuing with `execution()`.

See the example below, where we add 'mass' to a particle each timestep, based on a probablistic condition, and then split the particle once its 'mass' is larger than 5


In [ ]:
GrowingParticle = parcels.Particle.add_variables(
    [
        parcels.Variable("mass", initial=0),
        parcels.Variable("splittime", initial=-1),
        parcels.Variable("splitmass", initial=0),
    ]
)


def GrowParticles(particle, fieldset, time):
    import random

    # 25% chance per timestep for particle to grow
    if random.random() < 0.25:
        particle.mass += 1.0
    if (particle.mass >= 5.0) and (particle.splittime < 0):
        particle.splittime = time
        particle.splitmass = particle.mass / 2.0
        particle.mass = particle.mass / 2.0


pset = parcels.ParticleSet(fieldset=fieldset, pclass=GrowingParticle, lon=0, lat=0)
outfile = parcels.ParticleFile("growingparticles.zarr", pset, outputdt=1)

for t in range(40):
    pset.execute(
        GrowParticles, runtime=1, dt=1, output_file=outfile, verbose_progress=False
    )
    for p in pset:
        if p.splittime > 0:
            pset.add(
                parcels.ParticleSet(
                    fieldset=fieldset,
                    pclass=GrowingParticle,
                    lon=0,
                    lat=0,
                    time=p.splittime,
                    mass=p.splitmass,
                )
            )
            p.splittime = -1  # reset splittime

The 'trick' is that we place the `pset.execute()` call in a for-loop, so that we leave the Kernel-loop and can add Particles to the ParticleSet.

Indeed, if we plot the mass of particles as a function of time, we see that new particles are created every time a particle reaches a mass of 5.


In [ ]:
ds = xr.open_zarr("growingparticles.zarr")
plt.plot(ds.time.values[:].astype("timedelta64[s]").T, ds.mass.T)
plt.grid()
plt.xlabel("Time")
plt.ylabel("Particle 'mass'")
plt.show()